<a href="https://colab.research.google.com/github/pglez82/IFCB_semisupervised/blob/master/IFCB_FT_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load the data
We are going to finetune a resnet18 and extract features with it

In [1]:
import os

if not os.path.isfile("IFCB_data.tar") and not os.path.isdir("data"):
  print("Data do not exist in local. Downloading...")
  !wget -O IFCB_data.tar https://unioviedo-my.sharepoint.com/:u:/g/personal/gonzalezgpablo_uniovi_es/Ec2z0uC4lghEg-9MjzoJ9QkBK5n74QjS-LszB9dlNrPfaw?download=1
else:
  print("Data already exists. Skipping download.")

if not os.path.isdir("data"):
  print("Extracting the tar file...")
  !tar -xf "IFCB_data.tar"
  print("Done. Removing the tar file.")
  !rm -f IFCB_data.tar #Remove the original file to save space

Data do not exist in local. Downloading...
--2020-06-14 13:47:34--  https://unioviedo-my.sharepoint.com/:u:/g/personal/gonzalezgpablo_uniovi_es/Ec2z0uC4lghEg-9MjzoJ9QkBK5n74QjS-LszB9dlNrPfaw?download=1
Resolving unioviedo-my.sharepoint.com (unioviedo-my.sharepoint.com)... 13.107.136.9
Connecting to unioviedo-my.sharepoint.com (unioviedo-my.sharepoint.com)|13.107.136.9|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/gonzalezgpablo_uniovi_es/Documents/IFCB/data/IFCB_data.tar?&originalPath=aHR0cHM6Ly91bmlvdmllZG8tbXkuc2hhcmVwb2ludC5jb20vOnU6L2cvcGVyc29uYWwvZ29uemFsZXpncGFibG9fdW5pb3ZpX2VzL0VjMnowdUM0bGdoRWctOU1qem9KOVFrQks1bjc0UWpTLUxzekI5ZGxOclBmYXc_cnRpbWU9b2NtQWZta1EyRWc [following]
--2020-06-14 13:47:35--  https://unioviedo-my.sharepoint.com/personal/gonzalezgpablo_uniovi_es/Documents/IFCB/data/IFCB_data.tar?&originalPath=aHR0cHM6Ly91bmlvdmllZG8tbXkuc2hhcmVwb2ludC5jb20vOnU6L2cvcGVyc29uYWwvZ29uemFsZXpncGFibG9fdW5pb3ZpX2VzL0VjMnowdUM0bGdoRWctOU1q

# Download CSV with information about the images


In [2]:
import pandas as pd

if not os.path.isfile('IFCB.csv.zip'):
  print("CSV data do not exist. Downloading...")
  !wget -O IFCB.csv.zip "https://unioviedo-my.sharepoint.com/:u:/g/personal/gonzalezgpablo_uniovi_es/EfsVLhFsYJpPjO0KZlpWUq0BU6LaqJ989Re4XzatS9aG4Q?download=1"

data = pd.read_csv('IFCB.csv.zip',compression='infer', header=0,sep=',',quotechar='"')
print(data)

CSV data do not exist. Downloading...
--2020-06-14 14:01:12--  https://unioviedo-my.sharepoint.com/:u:/g/personal/gonzalezgpablo_uniovi_es/EfsVLhFsYJpPjO0KZlpWUq0BU6LaqJ989Re4XzatS9aG4Q?download=1
Resolving unioviedo-my.sharepoint.com (unioviedo-my.sharepoint.com)... 13.107.136.9
Connecting to unioviedo-my.sharepoint.com (unioviedo-my.sharepoint.com)|13.107.136.9|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/gonzalezgpablo_uniovi_es/Documents/IFCB/data/IFCB.csv.zip?&originalPath=aHR0cHM6Ly91bmlvdmllZG8tbXkuc2hhcmVwb2ludC5jb20vOnU6L2cvcGVyc29uYWwvZ29uemFsZXpncGFibG9fdW5pb3ZpX2VzL0Vmc1ZMaEZzWUpwUGpPMEtabHBXVXEwQlU2TGFxSjk4OVJlNFh6YXRTOWFHNFE_cnRpbWU9dlRwQ1ptc1EyRWc [following]
--2020-06-14 14:01:13--  https://unioviedo-my.sharepoint.com/personal/gonzalezgpablo_uniovi_es/Documents/IFCB/data/IFCB.csv.zip?&originalPath=aHR0cHM6Ly91bmlvdmllZG8tbXkuc2hhcmVwb2ludC5jb20vOnU6L2cvcGVyc29uYWwvZ29uemFsZXpncGFibG9fdW5pb3ZpX2VzL0Vmc1ZMaEZzWUpwUGpPMEtabHBXVXE

# Create training set

Here we make a reestructuration of the images depending on which class we consider

In [3]:
import progressbar
from tqdm import tqdm
tqdm.pandas()

classcolumn = "AutoClass" #Autoclass means 51 classes
yearstraining = ['2006'] #Years to consider as training
yearsvalidation = ['2007']
trainingfolder = "training"
validationfolder = "validation"

classes = pd.unique(data[classcolumn])
print("Considering %i classes" % len(classes))

print("Computing image paths...")
#Compute data paths
data['year'] = data['Sample'].str[6:10].astype(str)
data['path']="data"+'/'+data['year']+'/'+data['OriginalClass'].astype(str)+'/'+data['Sample'].astype(str)+'_'+data['roi_number'].apply(lambda x: str(x).zfill(5))+'.png'
print('Done')

if not os.path.isdir(trainingfolder):
  print("Create folder structure for training set...")
  os.mkdir(trainingfolder)
  for folder in classes:
    os.mkdir(os.path.join(trainingfolder,folder))
  print("Done.\nMoving images to the respective folders...")
  data[data['year'].isin(yearstraining)].progress_apply(lambda row: os.rename(row['path'],os.path.join(trainingfolder,row[classcolumn],os.path.basename(row['path']))),axis=1)
  print("Done")
else:
  print("Training data already there... Doing nothing")

if not os.path.isdir(validationfolder):
  print("Create folder structure for the validation set...")
  os.mkdir(validationfolder)
  for folder in classes:
    os.mkdir(os.path.join(validationfolder,folder))
  print("Done.\nMoving images to the respective folders...")
  data[data['year'].isin(yearsvalidation)].progress_apply(lambda row: os.rename(row['path'],os.path.join(validationfolder,row[classcolumn],os.path.basename(row['path']))),axis=1)
  print("Done")  
else:
  print("Validation data already there... Doing nothing")


Considering 51 classes
Computing image paths...
Done
Create folder structure for training set...
Done.
Moving images to the respective folders...


  0%|          | 425/273080 [00:00<01:04, 4246.88it/s]

Done
Create folder structure for the validation set...
Done.
Moving images to the respective folders...


100%|██████████| 273080/273080 [00:25<00:00, 10783.98it/s]

Done


# Configure the process

In [27]:
import torch

num_workers = 16 # @param
batch_size = 64 # @param 
train_dir = './training'
num_epochs_ft1 = 10 # @param
num_epochs_ft2 = 10 # @param

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using %s"%device)

Using cpu


# Prepare de DataLoaders for the CNN
In this step it is important to consider that we have to use images with the same size than the original network (so we can reuse the weights)

In [0]:
import torchvision
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

#Define transofrmations
train_transform = T.Compose([
  T.Resize(size=256),
  T.RandomResizedCrop(size=224),
  T.RandomHorizontalFlip(),
  T.ToTensor(),            
  T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])

#Define data loader
train_dset = ImageFolder(train_dir, transform=train_transform)
train_loader = DataLoader(train_dset,batch_size=batch_size,num_workers=num_workers,shuffle=True)


# Load the CNN

In [21]:
import torch.nn as nn

model = torchvision.models.resnet18(pretrained=True)
print("Adjusting the CNN for %s classes" % len(train_dset.classes))
num_classes = len(train_dset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)
#Define loss function
loss_fn = nn.CrossEntropyLoss()
model = model.to(device)

Adjusting the CNN for 51 classes


# Perform finetuning
First we only update the last layer for a few epochs, then we update all the weights with a small learning rate

In [32]:
import time

def run_epoch(model, loss_fn, loader, optimizer, device):
  """
  Train the model for one epoch.
  """
  start_time = time.time()
  # Set the model to training mode
  model.train()
  for step, (x, y) in enumerate(loader):
    
    x = x.to(device)
    y = y.to(device)

    # Run the model forward to compute scores and loss.
    scores = model(x)
    loss = loss_fn(scores, y)

    # Run the model backward and take a step using the optimizer.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if step % 5 == 0:
      spent = time.time()-start_time
      print(f"Step [{step}/{len(loader)}]\t Loss: {loss.item()} \t Time: {spent} secs [{(batch_size*5)/spent} ej/sec]]")
      start_time = time.time()

def check_accuracy(model, loader, device):
  """
  Check the accuracy of the model.
  """
  # Set the model to eval mode
  model.eval()
  num_correct, num_samples = 0, 0
  for x, y in loader:
    x = x.to(device)

    # Run the model forward, and compare the argmax score with the ground-truth
    # category.
    scores = model(x)
    _, preds = scores.data.cpu().max(1)
    num_correct += (preds == y).sum()
    num_examples += x.size(0)

  # Return the fraction of datapoints that were correctly classified.
  acc = float(num_correct) / num_examples
  return acc

for param in model.parameters():
  param.requires_grad = False
for param in model.fc.parameters():
  param.requires_grad = True

optimizer = torch.optim.Adam(model.fc.parameters(), lr=1e-3)

#First phase of finetuning
for epoch in range(num_epochs_ft1):
  # Run an epoch over the training data.
  print('Starting epoch %d / %d' % (epoch + 1,num_epochs_ft1))
  run_epoch(model, loss_fn, train_loader, optimizer, device)

  # Check accuracy on the train and val sets.
  train_acc = check_accuracy(model, train_loader, device)
  print('Train accuracy: ', train_acc)

#Allow updating all the weights in the second phase
for param in model.parameters():
  param.requires_grad = True

#Lower learning rate this time
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# Train the entire model for a few more epochs, checking accuracy on the
# train sets after each epoch.
for epoch in range(num_epochs_ft2):
  print('Starting epoch %d / %d' % (epoch + 1, num_epochs_ft2))
  run_epoch(model, loss_fn, train_loader, optimizer, dtype)

  train_acc = check_accuracy(model, train_loader, device)
  val_acc = check_accuracy(model, val_loader, device)
  print('Train accuracy: ', train_acc)



Starting epoch 1 / 10
Step [0/2047]	 Loss: 1.7301113605499268 	 Time: 18.878816843032837 secs [16.950214765079142 ej/sec]]
Step [5/2047]	 Loss: 1.6462788581848145 	 Time: 27.680025815963745 secs [11.560682859459192 ej/sec]]


KeyboardInterrupt: ignored